## VK groups content loading notebook

Here is the pipeline of how to load contents of groups

which are followed by friends of exact user(in example it`s me).

In [ ]:
from vkauth import VKAuth
import vk
import vk_tools as vt

import cv2
import numpy as np
import pandas as pd

from os.path import join

from time import sleep
from tqdm import tqdm_notebook as tqdm

In [ ]:
#authorization of app using modules imported.
app_id = '6203169'
perms = ['photos','friends','groups']
API_ver = '5.68'

Auth = VKAuth(perms, app_id, API_ver)
Auth.auth()

token = Auth.get_token()
user_id = Auth.get_user_id()

session = vk.Session(access_token=token)
api = vk.API(session)

In [ ]:
#id of user whose friends you gonna get, like: https://vk.com/id111111111
user_id = 111111111
friends_ids = vt.get_friends_ids(api, user_id)

In [ ]:
#collecting all groups 
groups = []

for i,friend in tqdm(enumerate(friends_ids)):
    if  i % 3 == 0:
        sleep(1)
    friend_groups = vt.get_user_groups(api, friend, moder=False)
    groups += friend_groups
    
print(len(groups))

In [ ]:
#converting groups to dataFrame
groups_df = pd.DataFrame(groups)
groups_df.drop_duplicates(inplace=True)
groups_df.head()

In [ ]:
data_path = '../Data/raw_data'

In [ ]:
#reading content(content == first 100 posts)
for i,group in tqdm(groups_df.iterrows()):
    name = group['name']
    group_id = group['id']
    #Different kinds of fails occures during scrapping
    #For examples there are names of groups with slashes
    #Like: 'The Kaaats / Indie-rock'
    try:
        content = vt.get_n_posts_text(api, group_id, n_posts=100)
        dst_path = join(data_path, name + '.txt')
        with open(dst_path, 'w+t') as f:
            f.write(content)
            
    except Exception as e:
        print('Error occured on group:', name)
        print(e)
        continue
    
    #need it because of requests limitaion in VK API.
    if  i % 3 == 0:
        sleep(1)    